In [3]:
# 데이터를 판다스를 통해 가공하고자 함
import pandas as pd
df =pd.read_csv('./dataset/train.csv')

In [71]:
#트레인 데이터의 형태 
df.head()

,Id,idhogar,parentesco1,hhsize,v2a1,hacdor,rooms,hacapo,v14a,refrig,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,21eb7fcc1,1,1,190000.0,0,3,0,1,1,...,100,1849,1,100,0,1.000000,0.0,100.0,1849,4
1,ID_f29eb3ddd,0e5d7a658,1,1,135000.0,0,4,0,1,1,...,144,4489,1,144,0,1.000000,64.0,144.0,4489,4
2,ID_68de51c94,2c7317ea8,1,1,NaN,0,8,0,1,1,...,121,8464,1,0,0,0.250000,64.0,121.0,8464,4
3,ID_d671db89c,2b58d945f,0,4,180000.0,0,5,0,1,1,...,81,289,16,121,4,1.777778,1.0,121.0,289,4
4,ID_d56d6f5f5,2b58d945f,0,4,180000.0,0,5,0,1,1,...,121,1369,16,121,4,1.777778,1.0,121.0,1369,4


In [18]:
#가구주인 사람들만 뽑은 데이터 프레임 df_h (parentesco1=1)
df_h = df.loc[df["parentesco1"]==1]
df_h.head()

,Id,idhogar,parentesco1,hhsize,v2a1,hacdor,rooms,hacapo,v14a,refrig,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,21eb7fcc1,1,1,190000.0,0,3,0,1,1,...,100,1849,1,100,0,1.000000,0.0,100.0,1849,4
1,ID_f29eb3ddd,0e5d7a658,1,1,135000.0,0,4,0,1,1,...,144,4489,1,144,0,1.000000,64.0,144.0,4489,4
2,ID_68de51c94,2c7317ea8,1,1,NaN,0,8,0,1,1,...,121,8464,1,0,0,0.250000,64.0,121.0,8464,4
5,ID_ec05b1a7b,2b58d945f,1,4,180000.0,0,5,0,1,1,...,121,1444,16,121,4,1.777778,1.0,121.0,1444,4
8,ID_1284f8aad,d6dae86b7,1,4,130000.0,1,2,0,1,1,...,81,900,16,81,4,16.000000,1.0,100.0,900,4


In [12]:
df_h.columns

Index(['Id', 'idhogar', 'parentesco1', 'hhsize', 'v2a1', 'hacdor', 'rooms',
       'hacapo', 'v14a', 'refrig',
       ...
       'SQBescolari', 'SQBage', 'SQBhogar_total', 'SQBedjefe', 'SQBhogar_nin',
       'SQBovercrowding', 'SQBdependency', 'SQBmeaned', 'agesq', 'Target'],
      dtype='object', length=145)

In [13]:
len(df_h)

2973

In [58]:
# 결과값 데이터에 해당하는 y를 뽑아냄 (타겟)
df_y =pd.DataFrame(df_h["Target"])
df_y.head()

,Target
0,4
1,4
2,4
5,4
8,4


In [51]:
#결과값을 포함하여 문자열을 포함하는 열을 제외시킨 피쳐값 df_x만들어준다
## no: edjefa, edjefe, dependency
df_x = df_h.drop(["Target", "edjefa", "edjefe", "dependency"],1)
df_x.head()

,parentesco1,hhsize,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,...,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq
0,1,1,190000.0,0,3,0,1,1,0,NaN,...,43,100,1849,1,100,0,1.000000,0.0,100.0,1849
1,1,1,135000.0,0,4,0,1,1,1,1.0,...,67,144,4489,1,144,0,1.000000,64.0,144.0,4489
2,1,1,NaN,0,8,0,1,1,0,NaN,...,92,121,8464,1,0,0,0.250000,64.0,121.0,8464
5,1,4,180000.0,0,5,0,1,1,1,1.0,...,38,121,1444,16,121,4,1.777778,1.0,121.0,1444
8,1,4,130000.0,1,2,0,1,1,0,NaN,...,30,81,900,16,81,4,16.000000,1.0,100.0,900


In [59]:
# 139개의 전체 피쳐로 softmax 돌림 
from tqdm import tqdm_notebook
import tensorflow as tf
import numpy as np

tf.set_random_seed(777)  # for reproducibility

# Predicting animal type based on various features
xy = df_h

x_data = df_x
y_data = df_y 

print(x_data.shape, y_data.shape)
print('\nx_data :\n', x_data)
print('\ny_data :\n', y_data)

nb_classes = 4  # 1 ~ 4

X = tf.placeholder(tf.float32, [None, 139])
Y = tf.placeholder(tf.int32, [None, 1])  # 1 ~ 4

Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot
print("\none_hot", Y_one_hot)

Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print("\nreshape", Y_one_hot)

W = tf.Variable(tf.random_normal([139, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                 labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

(2973, 139) (2973, 1)

x_data :
       parentesco1  hhsize      v2a1  hacdor  rooms  hacapo  v14a  refrig  \
0               1       1  190000.0       0      3       0     1       1   
1               1       1  135000.0       0      4       0     1       1   
2               1       1       NaN       0      8       0     1       1   
5               1       4  180000.0       0      5       0     1       1   
8               1       4  130000.0       1      2       0     1       1   
12              1       2  100000.0       0      3       0     1       1   
13              1       2       NaN       0      4       0     1       1   
18              1       4   90000.0       1      2       0     1       1   
20              1       2  215000.0       0      4       0     1       1   
21              1       2  150000.0       0      3       0     1       1   
23              1       3  100000.0       1      1       1     0       1   
26              1       1       NaN       0      5     

[2973 rows x 139 columns]

y_data :
       Target
0          4
1          4
2          4
5          4
8          4
12         4
13         4
18         4
20         4
21         4
23         4
26         4
27         4
30         4
33         4
39         4
40         4
43         4
45         4
49         4
53         2
56         4
58         2
59         4
61         4
64         2
68         4
72         4
74         4
77         4
...      ...
9477       2
9480       3
9481       4
9485       1
9487       4
9490       3
9492       2
9495       4
9499       4
9502       4
9505       4
9506       3
9507       4
9508       1
9510       4
9512       4
9515       1
9517       4
9519       4
9523       3
9526       4
9528       4
9530       4
9531       4
9534       3
9535       1
9541       2
9545       4
9551       2
9552       2

[2973 rows x 1 columns]

one_hot Tensor("one_hot_4:0", shape=(?, 1, 4), dtype=float32)

reshape Tensor("Reshape_13:0", shape=(?, 4), dtype=float32)


In [60]:
#학습시켰으나 loss값이 계속 nan으로 도출된다.
# Launch the graph in a session.
sess = tf.Session()

# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in range(2000):
    sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
    if step % 100 == 0:
        loss, acc = sess.run([cost, accuracy], feed_dict={
                             X: x_data, Y: y_data})
        print("Step: {:5}, \t Loss: {:.3f}, \t Acc: {:.2%}".format(
            step, loss, acc))

Step:     0, 	 Loss: nan, 	 Acc: 65.72%
Step:   100, 	 Loss: nan, 	 Acc: 65.72%
Step:   200, 	 Loss: nan, 	 Acc: 65.72%
Step:   300, 	 Loss: nan, 	 Acc: 65.72%
Step:   400, 	 Loss: nan, 	 Acc: 65.72%
Step:   500, 	 Loss: nan, 	 Acc: 65.72%
Step:   600, 	 Loss: nan, 	 Acc: 65.72%
Step:   700, 	 Loss: nan, 	 Acc: 65.72%
Step:   800, 	 Loss: nan, 	 Acc: 65.72%
Step:   900, 	 Loss: nan, 	 Acc: 65.72%
Step:  1000, 	 Loss: nan, 	 Acc: 65.72%
Step:  1100, 	 Loss: nan, 	 Acc: 65.72%
Step:  1200, 	 Loss: nan, 	 Acc: 65.72%
Step:  1300, 	 Loss: nan, 	 Acc: 65.72%
Step:  1400, 	 Loss: nan, 	 Acc: 65.72%
Step:  1500, 	 Loss: nan, 	 Acc: 65.72%
Step:  1600, 	 Loss: nan, 	 Acc: 65.72%
Step:  1700, 	 Loss: nan, 	 Acc: 65.72%
Step:  1800, 	 Loss: nan, 	 Acc: 65.72%
Step:  1900, 	 Loss: nan, 	 Acc: 65.72%


In [64]:
#불필요한 피쳐들 제거해 x1값 만들기 
# age, SQBescolari, SQBage, SQBhogar_total, SQBedjefe, SQBhogar_nin, SQBovercrowding,
# SQBdependency, SQBmeaned, agesq
df_x1 = df_h.drop(["Target", "edjefa", "edjefe", "dependency","age", 
                   "SQBescolari", "SQBage", "SQBhogar_total", "SQBedjefe", "SQBhogar_nin", "SQBovercrowding"],1)
df_x1.head()

,parentesco1,hhsize,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,...,lugar2,lugar3,lugar4,lugar5,lugar6,area1,area2,SQBdependency,SQBmeaned,agesq
0,1,1,190000.0,0,3,0,1,1,0,NaN,...,0,0,0,0,0,1,0,0.0,100.0,1849
1,1,1,135000.0,0,4,0,1,1,1,1.0,...,0,0,0,0,0,1,0,64.0,144.0,4489
2,1,1,NaN,0,8,0,1,1,0,NaN,...,0,0,0,0,0,1,0,64.0,121.0,8464
5,1,4,180000.0,0,5,0,1,1,1,1.0,...,0,0,0,0,0,1,0,1.0,121.0,1444
8,1,4,130000.0,1,2,0,1,1,0,NaN,...,0,0,0,0,0,1,0,1.0,100.0,900


In [67]:
#다시 돌려봄 :132개의 피쳐
from tqdm import tqdm_notebook
import tensorflow as tf
import numpy as np

tf.set_random_seed(777)  # for reproducibility

# Predicting animal type based on various features
xy = df_h

x_data = df_x1
y_data = df_y 

print(x_data.shape, y_data.shape)
print('\nx_data :\n', x_data)
print('\ny_data :\n', y_data)

nb_classes = 4  # 1 ~ 4

X = tf.placeholder(tf.float32, [None, 132])
Y = tf.placeholder(tf.int32, [None, 1])  # 1 ~ 4

Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot
print("\none_hot", Y_one_hot)

Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print("\nreshape", Y_one_hot)

W = tf.Variable(tf.random_normal([132, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                 labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

(2973, 132) (2973, 1)

x_data :
       parentesco1  hhsize      v2a1  hacdor  rooms  hacapo  v14a  refrig  \
0               1       1  190000.0       0      3       0     1       1   
1               1       1  135000.0       0      4       0     1       1   
2               1       1       NaN       0      8       0     1       1   
5               1       4  180000.0       0      5       0     1       1   
8               1       4  130000.0       1      2       0     1       1   
12              1       2  100000.0       0      3       0     1       1   
13              1       2       NaN       0      4       0     1       1   
18              1       4   90000.0       1      2       0     1       1   
20              1       2  215000.0       0      4       0     1       1   
21              1       2  150000.0       0      3       0     1       1   
23              1       3  100000.0       1      1       1     0       1   
26              1       1       NaN       0      5     

[2973 rows x 132 columns]

y_data :
       Target
0          4
1          4
2          4
5          4
8          4
12         4
13         4
18         4
20         4
21         4
23         4
26         4
27         4
30         4
33         4
39         4
40         4
43         4
45         4
49         4
53         2
56         4
58         2
59         4
61         4
64         2
68         4
72         4
74         4
77         4
...      ...
9477       2
9480       3
9481       4
9485       1
9487       4
9490       3
9492       2
9495       4
9499       4
9502       4
9505       4
9506       3
9507       4
9508       1
9510       4
9512       4
9515       1
9517       4
9519       4
9523       3
9526       4
9528       4
9530       4
9531       4
9534       3
9535       1
9541       2
9545       4
9551       2
9552       2

[2973 rows x 1 columns]

one_hot Tensor("one_hot_6:0", shape=(?, 1, 4), dtype=float32)

reshape Tensor("Reshape_21:0", shape=(?, 4), dtype=float32)


In [68]:
#이전과 같은 결과 -> 피쳐값들간의 normalized의 필요성 인지 
# Launch the graph in a session.
sess = tf.Session()

# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in range(2000):
    sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
    if step % 100 == 0:
        loss, acc = sess.run([cost, accuracy], feed_dict={
                             X: x_data, Y: y_data})
        print("Step: {:5}, \t Loss: {:.3f}, \t Acc: {:.2%}".format(
            step, loss, acc))

Step:     0, 	 Loss: nan, 	 Acc: 65.72%
Step:   100, 	 Loss: nan, 	 Acc: 65.72%
Step:   200, 	 Loss: nan, 	 Acc: 65.72%
Step:   300, 	 Loss: nan, 	 Acc: 65.72%
Step:   400, 	 Loss: nan, 	 Acc: 65.72%
Step:   500, 	 Loss: nan, 	 Acc: 65.72%
Step:   600, 	 Loss: nan, 	 Acc: 65.72%
Step:   700, 	 Loss: nan, 	 Acc: 65.72%
Step:   800, 	 Loss: nan, 	 Acc: 65.72%
Step:   900, 	 Loss: nan, 	 Acc: 65.72%
Step:  1000, 	 Loss: nan, 	 Acc: 65.72%
Step:  1100, 	 Loss: nan, 	 Acc: 65.72%
Step:  1200, 	 Loss: nan, 	 Acc: 65.72%
Step:  1300, 	 Loss: nan, 	 Acc: 65.72%
Step:  1400, 	 Loss: nan, 	 Acc: 65.72%
Step:  1500, 	 Loss: nan, 	 Acc: 65.72%
Step:  1600, 	 Loss: nan, 	 Acc: 65.72%
Step:  1700, 	 Loss: nan, 	 Acc: 65.72%
Step:  1800, 	 Loss: nan, 	 Acc: 65.72%
Step:  1900, 	 Loss: nan, 	 Acc: 65.72%


In [72]:
df_h.head()

,parentesco1,hhsize,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,1,1,190000.0,0,3,0,1,1,0,NaN,...,100,1849,1,100,0,1.000000,0.0,100.0,1849,4
1,1,1,135000.0,0,4,0,1,1,1,1.0,...,144,4489,1,144,0,1.000000,64.0,144.0,4489,4
2,1,1,NaN,0,8,0,1,1,0,NaN,...,121,8464,1,0,0,0.250000,64.0,121.0,8464,4
5,1,4,180000.0,0,5,0,1,1,1,1.0,...,121,1444,16,121,4,1.777778,1.0,121.0,1444,4
8,1,4,130000.0,1,2,0,1,1,0,NaN,...,81,900,16,81,4,16.000000,1.0,100.0,900,4


In [83]:
df_sanitario = df_h[['sanitario1','sanitario2','sanitario3','sanitario5','sanitario6']]
df_sanitario.head()
df_sanitario[]

,sanitario1,sanitario2,sanitario3,sanitario5,sanitario6
0,0,1,0,0,0
1,0,1,0,0,0
2,0,1,0,0,0
5,0,1,0,0,0
8,0,1,0,0,0


In [ ]:
df_sanitario["sanitario1"]

def numbering():
    colstr

In [79]:
df_sanitario.columns

Index(['sanitario1', 'sanitario2', 'sanitario3', 'sanitario5', 'sanitario6'], dtype='object')

In [92]:
df_sanitario["sanitario"]=0

C:\python\Anaconda3-52\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [93]:
df_sanitario.head()

,sanitario1,sanitario2,sanitario3,sanitario5,sanitario6,sanitario
0,0,1,0,0,0,0
1,0,1,0,0,0,0
2,0,1,0,0,0,0
5,0,1,0,0,0,0
8,0,1,0,0,0,0


In [103]:
df_sanitario["sanitario"]=df["sanitario1"]*1+df["sanitario2"]*2+df["sanitario3"]*3+df["sanitario5"]*5+df["sanitario6"]*6

C:\python\Anaconda3-52\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [105]:
df_sanitario.head()

,sanitario1,sanitario2,sanitario3,sanitario5,sanitario6,sanitario
0,0,1,0,0,0,2
1,0,1,0,0,0,2
2,0,1,0,0,0,2
5,0,1,0,0,0,2
8,0,1,0,0,0,2


In [110]:
df_sanitario.drop(["sanitario1","sanitario2","sanitario3","sanitario5","sanitario6"],1)

,sanitario
0,2
1,2
2,2
5,2
8,2
12,2
13,2
18,2
20,2
21,2
